<br>
<h1 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">XGBoost + Optuna 🔥</h1>
<br>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/25225/logos/header.png?t=2021-01-27-17-34-26)

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Import Required Libraries 📚</h1>

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

import optuna
from optuna import Trial, visualization

import warnings
warnings.filterwarnings("ignore")

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Read the Data 📖</h1>

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
feature_cols = [col for col in train.columns.tolist() if col not in ['id', 'target']]
target_col = ['target']

In [ ]:
class_dict = {
    'Class_1': 0,
    'Class_2': 1,
    'Class_3': 2,
    'Class_4': 3,
}

In [ ]:
train['target'] = train['target'].map(class_dict)

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Folds</h1>

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for i, (trn, val) in enumerate(skf.split(train, train[target_col])):
    train.loc[val, 'kfold'] = i
train['kfold'] = train['kfold'].astype(int)

In [ ]:
def fit_xgb(trial, xtr, ytr, xval, yval):
    params = {
        "n_estimators": trial.suggest_int("n_estimators",200,2000,100),
        "subsample": trial.suggest_discrete_uniform("subsample",0.6,1,0.1),
        "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree",0.6,1,0.1),
        "eta": trial.suggest_loguniform("eta",1e-3,0.1),
        "reg_alpha": trial.suggest_int("reg_alpha",1,50),
        "reg_lambda": trial.suggest_int("reg_lambda",5,100),
        "max_depth": trial.suggest_int("max_depth",5,20),
        "min_child_weight": trial.suggest_int("min_child_weight",5,20),
    }
    
    model = xgb.XGBClassifier(**params, tree_method='gpu_hist', random_state=42)
    model.fit(xtr, ytr.reshape(-1,), eval_metric='mlogloss')
    
    y_val_pred = model.predict_proba(xval)
    
    log = {
        "train logloss": log_loss(ytr, model.predict_proba(xtr)),
        "valid logloss": log_loss(yval, y_val_pred)
    }
    
    return model, log

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Objective Function</h1>

In [ ]:
def objective(trial):
    loss = 0
    for fold in range(5):
        trn_idx = train['kfold'] != fold
        val_idx = train['kfold'] == fold
        trn = train.loc[trn_idx, :]
        val = train.loc[val_idx, :]

        xtr, ytr = trn[feature_cols].values, trn[target_col].values
        xval, yval = val[feature_cols].values, val[target_col].values
        
        model, log = fit_xgb(trial, xtr, ytr, xval, yval)
        loss += log['valid logloss']/5
        
    return loss

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Optimize 📈</h1>

In [ ]:
study = optuna.create_study(direction="minimize", study_name='Xgboost optimization')
study.optimize(objective, n_trials=20)

In [ ]:
history = study.trials_dataframe()
history.sort_values(by="value", ascending=True)

In [ ]:
study.best_params

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Visualizations 📊</h1>

In [ ]:
visualization.plot_slice(study, target_name = 'Average Validation LogLoss')

In [ ]:
visualization.plot_optimization_history(study, target_name = 'Average Validation LogLoss')

In [ ]:
visualization.plot_parallel_coordinate(study, target_name = 'Average Validation LogLoss')

In [ ]:
visualization.plot_param_importances(study, target_name = 'Average Validation LogLoss')

<h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Re-train on the Entire Data</h1>

In [ ]:
clf = xgb.XGBClassifier(**study.best_params, tree_method='gpu_hist', random_state=42)
clf.fit(train[feature_cols], train[target_col], eval_metric='mlogloss')

In [ ]:
test_predictions = clf.predict_proba(test[feature_cols])

In [ ]:
predictions_df = pd.DataFrame(test_predictions, columns = ["Class_1", "Class_2", "Class_3", "Class_4"])
predictions_df['id'] = sample['id']

In [ ]:
predictions_df.to_csv("submission.csv", index = False)

![Upvote!](https://img.shields.io/badge/Upvote-If%20you%20like%20my%20work-07b3c8?style=for-the-badge&logo=kaggle)